<a href="https://colab.research.google.com/github/Shreyansh0906/ASSIGNMENT-2-G24AI2029-SHREYANSH-KUMAR-FDS/blob/main/music_reco_Oct24_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pysqlite3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.7/40.7 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pysqlite3: filename=pysqlite3-0.5.4-cp312-cp312-linux_x86_64.whl size=167808 sha256=e838c30cb8c99dd62cd674a5c85ab561b57fd4191db2f23366c3c65a0527781a
  Stored in directory: /root/.cache/pip/wheels/5f/02/a6/284262355044b549376150fa1166bbd8106585da0c15280db3
Successfully built pysqlite3


In [2]:
from pysqlite3 import dbapi2 as sqlite3
print(sqlite3.sqlite_version_info)

(3, 37, 2)


In [3]:
import pandas as pd
from IPython.display import display, HTML

In [4]:
dbname = "music_streaming_1.db"

def printSqlResults(cursor, tbl):
  df=pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
  display(HTML("<b><font color=Green>"+tbl+"</font></b>"+df.to_html(index=False)))

def runSql(caption, query):
  conn = sqlite3.connect(dbname)
  cursor=conn.cursor()
  cursor.execute(query)
  printSqlResults(cursor,caption)
  conn.close()

In [5]:
conn = sqlite3.connect(dbname)
cursor = conn.cursor()

cursor.execute("""
      CREATE TABLE IF NOT EXISTS Users (
        user_id INTEGER PRIMARY KEY,
        name VARCHAR(100) NOT NULL,
        email VARCHAR(100) NOT NULL UNIQUE
      );
""")

cursor.execute("""
  CREATE TABLE IF NOT EXISTS Songs (
    song_id INTEGER PRIMARY KEY,
    title VARCHAR(100) NOT NULL,
    artist VARCHAR(100) NOT NULL,
    genre VARCHAR(100)
  );
"""
)

cursor.execute("""
    CREATE TABLE IF NOT EXISTS Listens (
      listen_id INTEGER PRIMARY KEY,
      user_id INTEGER NOT NULL,
      song_id INTEGER NOT NULL,
      rating FLOAT,
      listen_time TIMESTAMP,
      FOREIGN KEY (user_id) REFERENCES Users(user_id),
      FOREIGN KEY (song_id) REFERENCES Songs(song_id)
    );
""")

cursor.execute("""
  CREATE TABLE IF NOT EXISTS Recommendations (
      recommendation_id INTEGER NOT NULL,
      recommendation_time TIMESTAMP,
      user_id INTEGER NOT NULL,
      song_id INTEGER NOT NULL,
      FOREIGN KEY (user_id) REFERENCES Users(user_id),
      FOREIGN KEY (song_id) REFERENCES Songs(song_id)
  );
""")

conn.commit()
conn.close()

In [6]:
conn = sqlite3.connect(dbname)
sqlite3.enable_callback_tracebacks(True)

cursor=conn.cursor()
cursor.execute("delete from Songs;")
cursor.execute("delete from Users;")
cursor.execute("delete from Listens;")
cursor.execute("delete from Recommendations;")

#Insert data into the table

cursor.execute("""
  INSERT INTO Users (user_id, name, email)
  VALUES
    (1, 'Mickey', 'mickey@example.com'),
    (2, 'Minnie', 'minnie@example.com'),
    (3, 'Daffy', 'daffy@example.com'),
    (4, 'Pluto', 'pluto@example.com');
""")
# Insert sample songs from Taylor Swift, Ed Sheeran, Beatles
cursor.execute("""
INSERT INTO Songs (song_id, title, artist, genre)
VALUES
    (1, 'Evermore', 'Taylor Swift', 'Pop'),
    (2, 'Willow', 'Taylor Swift', 'Pop'),
    (3, 'Shape of You', 'Ed Sheeran', 'Rock'),
    (4, 'Photograph', 'Ed Sheeran', 'Rock'),
    (5, 'Shivers', 'Ed Sheeran', 'Rock'),
    (6, 'Yesterday', 'Beatles', 'Classic'),
    (7, 'Yellow Submarine', 'Beatles', 'Classic'),
    (8, 'Hey Jude', 'Beatles', 'Classic'),
    (9, 'Bad Blood', 'Taylor Swift', 'Rock'),
    (10, 'DJ Mix', 'DJ', NULL);
""")

# Insert sample listens
cursor.execute("""
INSERT INTO Listens (listen_id, user_id, song_id, rating, listen_time)
VALUES
    (1, 1, 1, 4.5, '2024-08-30 14:35:00'),
    (2, 1, 2, 4.2, NULL),
    (3, 1, 6, 3.9, '2024-08-29 10:15:00'),
    (4, 2, 2, 4.7, NULL),
    (5, 2, 7, 4.6, '2024-08-28 09:20:00'),
    (6, 2, 8, 3.9, '2024-08-27 16:45:00'),
    (7, 3, 1, 2.9, NULL),
    (8, 3, 2, 4.9, '2024-08-26 12:30:00'),
    (9, 3, 6, NULL, NULL);
""")

# Commit changes and close the connection
conn.commit()
conn.close()

runSql('Users', "select * from Users;")
runSql('Songs', "select * from Songs;")
runSql('Listens', "select * from Listens;")

user_id,name,email
1,Mickey,mickey@example.com
2,Minnie,minnie@example.com
3,Daffy,daffy@example.com
4,Pluto,pluto@example.com


song_id,title,artist,genre
1,Evermore,Taylor Swift,Pop
2,Willow,Taylor Swift,Pop
3,Shape of You,Ed Sheeran,Rock
4,Photograph,Ed Sheeran,Rock
5,Shivers,Ed Sheeran,Rock
6,Yesterday,Beatles,Classic
7,Yellow Submarine,Beatles,Classic
8,Hey Jude,Beatles,Classic
9,Bad Blood,Taylor Swift,Rock
10,DJ Mix,DJ,None


listen_id,user_id,song_id,rating,listen_time
1,1,1,4.5,2024-08-30 14:35:00
2,1,2,4.2,None
3,1,6,3.9,2024-08-29 10:15:00
4,2,2,4.7,None
5,2,7,4.6,2024-08-28 09:20:00
6,2,8,3.9,2024-08-27 16:45:00
7,3,1,2.9,None
8,3,2,4.9,2024-08-26 12:30:00
9,3,6,NaN,None


In [7]:
qry_classic_songs = """
SELECT Songs.title, Songs.artist FROM Songs
WHERE Songs.genre= 'Classic';
"""
runSql('Classic Query', qry_classic_songs)

title,artist
Yesterday,Beatles
Yellow Submarine,Beatles
Hey Jude,Beatles


In [8]:
#Find titles and artists if songs in Classic Genre with titles starting like Ye...

qry_classic_songs2 = """
SELECT Songs.title, Songs.artist FROM Songs
WHERE Songs.genre= 'Classic' AND
  Songs.title LIKE 'Ye%';
"""
runSql('Classic Query with songs like Ye..', qry_classic_songs2)

title,artist
Yesterday,Beatles
Yellow Submarine,Beatles


In [9]:
#List all the genres in the Songs table
qry_genres = """
SELECT genre from Songs;
"""
runSql('All genres', qry_genres)

genre
Pop
Pop
Rock
Rock
Rock
Classic
Classic
Classic
Rock
None


In [10]:
#Find the unique genres from the Songs table

qry_distinct = """
SELECT DISTINCT genre
FROM Songs;
"""

runSql('Distinct Genres', qry_distinct)

genre
Pop
Rock
Classic
None


In [11]:
#Find the number of songs by Taylor Swift in different genres

qry_taylor_count="""
SELECT artist, genre,count(*) as num_songs
FROM Songs
WHERE Songs.artist='Taylor Swift'
GROUP BY artist, genre;
"""
runSql('Count of songs by Taylor Swift in different genres', qry_taylor_count)

artist,genre,num_songs
Taylor Swift,Pop,2
Taylor Swift,Rock,1


In [12]:
#Find the number of songs by all artists in different genres

qry_artist_count="""
SELECT artist, genre,count(*) as num_songs
FROM Songs
GROUP BY artist, genre;
"""
runSql('Count of songs by Taylor Swift in different genres', qry_artist_count)


artist,genre,num_songs
Beatles,Classic,3
DJ,None,1
Ed Sheeran,Rock,3
Taylor Swift,Pop,2
Taylor Swift,Rock,1


In [13]:
#Create one single table for songs and listens. Redundancies are allowed

qry_large_table = """
SELECT * FROM Songs
LEFT JOIN Listens ON Songs.song_id=Listens.song_id
LEFT JOIN Users ON Listens.user_id=Users.user_id;
"""
runSql('One Large Table',qry_large_table)

song_id,title,artist,genre,listen_id,user_id,song_id,rating,listen_time,user_id,name,email
1,Evermore,Taylor Swift,Pop,1.0,1.0,1.0,4.5,2024-08-30 14:35:00,1.0,Mickey,mickey@example.com
1,Evermore,Taylor Swift,Pop,7.0,3.0,1.0,2.9,None,3.0,Daffy,daffy@example.com
2,Willow,Taylor Swift,Pop,2.0,1.0,2.0,4.2,None,1.0,Mickey,mickey@example.com
2,Willow,Taylor Swift,Pop,4.0,2.0,2.0,4.7,None,2.0,Minnie,minnie@example.com
2,Willow,Taylor Swift,Pop,8.0,3.0,2.0,4.9,2024-08-26 12:30:00,3.0,Daffy,daffy@example.com
3,Shape of You,Ed Sheeran,Rock,NaN,NaN,NaN,NaN,None,NaN,None,None
4,Photograph,Ed Sheeran,Rock,NaN,NaN,NaN,NaN,None,NaN,None,None
5,Shivers,Ed Sheeran,Rock,NaN,NaN,NaN,NaN,None,NaN,None,None
6,Yesterday,Beatles,Classic,3.0,1.0,6.0,3.9,2024-08-29 10:15:00,1.0,Mickey,mickey@example.com
6,Yesterday,Beatles,Classic,9.0,3.0,6.0,NaN,None,3.0,Daffy,daffy@example.com


In [14]:
#Find the Highly Rates Songs. Consider Rating above 4.6 as highly rated

qry_rating_songs="""
SELECT Songs.song_id, Songs.title, Songs.artist, Listens.rating
FROM Songs
JOIN Listens ON Songs.song_id = Listens.song_id
WHERE Listens.rating > 4.6
"""
runSql('Highly Rated Songs',qry_rating_songs)

#Find tha average rating of every song

qry_avg_songs="""
SELECT Songs.song_id, Songs.title, Songs.artist, AVG(Listens.rating)
FROM Songs
JOIN Listens ON Songs.song_id = Listens.song_id
GROUP BY Songs.song_id, Songs.title, Songs.artist
"""
runSql('Highly Rated Songs',qry_avg_songs)



song_id,title,artist,rating
2,Willow,Taylor Swift,4.7
2,Willow,Taylor Swift,4.9


song_id,title,artist,AVG(Listens.rating)
1,Evermore,Taylor Swift,3.7
2,Willow,Taylor Swift,4.6
6,Yesterday,Beatles,3.9
7,Yellow Submarine,Beatles,4.6
8,Hey Jude,Beatles,3.9


In [15]:
#Find the popular songs by counting the listens

qry_pop_songs = """
SELECT Songs.song_id, Songs.title, Songs.artist, count(Listens.song_id)
FROM Songs
JOIN Listens
ON Songs.song_id=Listens.song_id
GROUP BY Songs.title, Songs.artist
ORDER BY COUNT(Listens.song_id) DESC;
"""

runSql('Popular Songs', qry_pop_songs)

song_id,title,artist,count(Listens.song_id)
2,Willow,Taylor Swift,3
1,Evermore,Taylor Swift,2
6,Yesterday,Beatles,2
8,Hey Jude,Beatles,1
7,Yellow Submarine,Beatles,1


In [16]:
#Find songs by Ed Sheeran and Taylor Swift

qry_edtaylor="""
SELECT title, artist
FROM Songs
WHERE artist IN ('Ed Sheeran', 'Taylor Swift');
"""
runSql('Songs by Ed Sheeran and Taylor Swift', qry_edtaylor)

#Find songs from both pop and rock genres

qry_popandrock = """
SELECT title, artist
FROM Songs
WHERE genre='Pop'
UNION
SELECT title, artist
FROM Songs
WHERE genre='Rock'
"""
runSql('Songs on both Pop and Rock', qry_popandrock)


title,artist
Evermore,Taylor Swift
Willow,Taylor Swift
Shape of You,Ed Sheeran
Photograph,Ed Sheeran
Shivers,Ed Sheeran
Bad Blood,Taylor Swift


title,artist
Bad Blood,Taylor Swift
Evermore,Taylor Swift
Photograph,Ed Sheeran
Shape of You,Ed Sheeran
Shivers,Ed Sheeran
Willow,Taylor Swift


In [17]:
#Find songs listened to by user_id 1

qry_listen_usr="""
SELECT title, artist
FROM Songs
WHERE song_id IN (SELECT song_id FROM Listens WHERE Listens.listen_time IS NULL);
"""
runSql('Songs listened by user_id=1',qry_listen_usr)

title,artist
Evermore,Taylor Swift
Willow,Taylor Swift
Yesterday,Beatles


In [18]:
#Computing the recommendations
#Find the song pairs which are shared across > 1 user

qry_sharing="""
WITH song_similarity AS (
SELECT u1.song_id as song1, u2.song_id as song2, COUNT(*) AS common_users
FROM LISTENS u1
JOIN LISTENS u2
ON u1.user_id=u2.user_id
AND u1.song_id != u2.song_id
GROUP BY u1.song_id, u2.song_id
HAVING COUNT(*)>1
),

recs AS (
  SELECT user_id, song2 as song_id
  FROM song_similarity
  JOIN Listens as L
  ON L.song_id = song_similarity.song1
  WHERE song_similarity.song2 NOT IN
  (SELECT song_id FROM Listens as temp where temp.user_id=L.user_id)
)
select * from recs;
"""
runSql('Songs shared', qry_sharing)

user_id,song_id
2,1
2,6


In [19]:
#ASSIGNMENT-2. To be submitted as a PDF with queries and screenshots.

#1 Insert the above into the recommendations table
#2 Generate the recommendaions for Minnie

#3 Re-do the generation of recommendations now on the basis of listen time
#4 Generate new recommendations
#5 What are the differences with the static method on #2 above

In [20]:
# Task 1: Insert the recommendations into the database
qry_sharing = """
WITH song_similarity AS (
  SELECT
    u1.song_id as song1,
    u2.song_id as song2,
    COUNT(*) AS common_users
  FROM LISTENS u1
  JOIN LISTENS u2
    ON u1.user_id = u2.user_id AND u1.song_id != u2.song_id
  GROUP BY
    u1.song_id,
    u2.song_id
  HAVING
    COUNT(*) > 1
),
recs AS (
  SELECT
    L.user_id,
    song_similarity.song2 as song_id
  FROM song_similarity
  JOIN Listens as L
    ON L.song_id = song_similarity.song1
  WHERE
    song_similarity.song2 NOT IN (
      SELECT song_id FROM Listens as temp WHERE temp.user_id = L.user_id
    )
)
SELECT * FROM recs;
"""

qry_insert_recommendations = f"""
INSERT INTO Recommendations (recommendation_id, recommendation_time, user_id, song_id)
SELECT
    (SELECT IFNULL(MAX(recommendation_id), 0) FROM Recommendations) + ROW_NUMBER() OVER () as recommendation_id,
    CURRENT_TIMESTAMP,
    user_id,
    song_id
FROM ({qry_sharing.replace(';', '')});
"""

conn = sqlite3.connect(dbname)
cursor = conn.cursor()
cursor.execute("DELETE FROM Recommendations;")
cursor.execute(qry_insert_recommendations)
conn.commit()
conn.close()

print("--- Task 1 Complete: Recommendations have been inserted. ---")


# Task 2: Generate recommendations for Minnie
qry_minnie_recs = """
SELECT
    U.name,
    S.title as recommended_song,
    S.artist
FROM Recommendations AS R
JOIN Users AS U ON R.user_id = U.user_id
JOIN Songs AS S ON R.song_id = S.song_id
WHERE U.name = 'Minnie';
"""
runSql("Recommendations for Minnie (Static Method)", qry_minnie_recs)

--- Task 1 Complete: Recommendations have been inserted. ---


name,recommended_song,artist
Minnie,Evermore,Taylor Swift
Minnie,Yesterday,Beatles


In [21]:
# Task 3 & 4: Re-do recommendations based on listen time
qry_sharing_by_time = """
WITH song_similarity_by_time AS (
  SELECT
    l1.song_id AS song1,
    l2.song_id AS song2
  FROM Listens l1
  JOIN Listens l2 ON l1.user_id = l2.user_id
  WHERE
    l1.song_id != l2.song_id
    AND l1.listen_time IS NOT NULL
    AND l2.listen_time IS NOT NULL
    AND ABS(strftime('%s', l1.listen_time) - strftime('%s', l2.listen_time)) < 7200 -- 2-hour window
  GROUP BY
    l1.song_id,
    l2.song_id
),
recs_by_time AS (
  SELECT DISTINCT
    L.user_id,
    S.song2 AS song_id
  FROM song_similarity_by_time AS S
  JOIN Listens AS L ON L.song_id = S.song1
  WHERE S.song2 NOT IN (
    SELECT song_id FROM Listens temp WHERE temp.user_id = L.user_id
  )
)
SELECT * FROM recs_by_time;
"""
runSql('New Recommendations Based on Listen Time', qry_sharing_by_time)

user_id,song_id


### Task 5: Comparison of Methods

1.  **Results:**
    * The **static method** recommended two songs for Minnie: "Evermore" and "Yesterday".
    * The **listen-time method** produced **no recommendations** for any user.

2.  **Reason for the Difference:**
    * The **static method** looks at a user's entire listening history. It identifies broad, long-term similarities in taste. It worked because it found general overlaps between users' song preferences over all time.
    * The **listen-time method** is stricter and looks for songs listened to in the same session (within a 2-hour window). It tries to find contextual connections. In our dataset, no user has a recorded history of listening to two different songs so close together, so no "in-session" pairs were found, and the algorithm could not make any recommendations.